In [3]:
import os
import nltk

In [4]:
corpus_path=os.path.abspath("../data/NKJP_1.2_nltk/")
corpus=nltk.corpus.reader.TaggedCorpusReader(root=corpus_path, fileids=".*")

In [31]:
tags = [x[1] for x in corpus.tagged_words()]
fqd = nltk.FreqDist(tags)



In [9]:
fqd.most_common()

[('INTERP', 223499),
 ('QUB', 68402),
 ('CONJ', 44953),
 ('FIN:SG:TER:IMPERF', 30242),
 ('SUBST:SG:GEN:F', 29592),
 ('PREP:LOC:NWOK', 27118),
 ('SUBST:SG:NOM:M1', 23810),
 ('SUBST:SG:GEN:M3', 23296),
 ('SUBST:SG:NOM:F', 22481),
 ('ADV:POS', 21231),
 ('COMP', 17830),
 ('PREP:GEN', 17810),
 ('ADV', 17299),
 ('PREP:ACC', 17253),
 ('PREP:LOC', 15944),
 ('SUBST:SG:ACC:F', 15543),
 ('SUBST:SG:NOM:M3', 15161),
 ('SUBST:SG:LOC:M3', 14041),
 ('SUBST:SG:ACC:M3', 14034),
 ('PREP:GEN:NWOK', 13583),
 ('SUBST:SG:GEN:N', 13415),
 ('SUBST:SG:LOC:F', 12290),
 ('SUBST:SG:NOM:N', 11814),
 ('PRAET:SG:M1:PERF', 11078),
 ('SUBST:SG:ACC:N', 10673),
 ('INF:PERF', 10521),
 ('PREP:INST:NWOK', 10392),
 ('ADJ:SG:NOM:F:POS', 10094),
 ('SUBST:PL:GEN:M3', 9921),
 ('ADJ:SG:GEN:F:POS', 9804),
 ('FIN:PL:TER:IMPERF', 9599),
 ('SUBST:PL:GEN:F', 9478),
 ('ADJ:SG:GEN:M3:POS', 9444),
 ('SUBST:SG:GEN:M1', 9343),
 ('INF:IMPERF', 8698),
 ('BREV:PUN', 8277),
 ('PRAET:SG:M1:IMPERF', 7818),
 ('ADJ:SG:NOM:M3:POS', 7625),
 ('PREP:A

In [54]:
# @TODO merge tags so that their cardinalities are ≥  100, 1000
sets = fqd.most_common()
sets = [(s[0].split(':'), s[1]) for s in sets]
sets = [(s[0][0], s[0][1:], s[1]) for s in sets]
# sets = {s[0][0]:{'tags':s[0][1:], 'card': s[1]} for s in sets}
# we want a struct:
# main_tag : {
# list(
#     {tags: [other tags], card: cardinality}
# )
# }
structurized_data = {}
for s in sets:
    k = s[0]
    v = {'tags': s[1], 'card': s[2]}
    if k in structurized_data.keys():
        structurized_data[k]+=[v]
    else:
        structurized_data[k] =[v]
        
structurized_data

{'INTERP': [{'tags': [], 'card': 223499}],
 'QUB': [{'tags': [], 'card': 68402},
  {'tags': ['NWOK'], 'card': 35},
  {'tags': ['WOK'], 'card': 31}],
 'CONJ': [{'tags': [], 'card': 44953}],
 'FIN': [{'tags': ['SG', 'TER', 'IMPERF'], 'card': 30242},
  {'tags': ['PL', 'TER', 'IMPERF'], 'card': 9599},
  {'tags': ['SG', 'PRI', 'IMPERF'], 'card': 7302},
  {'tags': ['SG', 'TER', 'PERF'], 'card': 3860},
  {'tags': ['PL', 'PRI', 'IMPERF'], 'card': 2598},
  {'tags': ['SG', 'SEC', 'IMPERF'], 'card': 2426},
  {'tags': ['PL', 'TER', 'PERF'], 'card': 1219},
  {'tags': ['SG', 'PRI', 'PERF'], 'card': 939},
  {'tags': ['PL', 'PRI', 'PERF'], 'card': 573},
  {'tags': ['SG', 'SEC', 'PERF'], 'card': 382},
  {'tags': ['PL', 'SEC', 'IMPERF'], 'card': 297},
  {'tags': ['PL', 'SEC', 'PERF'], 'card': 61}],
 'SUBST': [{'tags': ['SG', 'GEN', 'F'], 'card': 29592},
  {'tags': ['SG', 'NOM', 'M1'], 'card': 23810},
  {'tags': ['SG', 'GEN', 'M3'], 'card': 23296},
  {'tags': ['SG', 'NOM', 'F'], 'card': 22481},
  {'tags'

We see that:
QUB has 1 optional tag, we'll want to remove it since they make a really small difference.  
PREP has 1 optional tag, but the sizes are ok, so we'll ignore it  
ADV is good as it is  
PRAET has 1 optional tag  
PPRON12 has 1 optional tag  

why am I mentioning this?
because the missing tags were breaking my naive algorithm lol.

In [65]:
# remove optional tags - in most cases their respective set sizes are too small anyway
keys_with_optional_tags = {'PREP', 'PRAET', 'PPRON12'}

def get_num_of_mandatory_tags(A):
    min_len = 0
    for el in A:
        if(len(el['tags'])<min_len):
            min_len = len(el['tags'])
    
    return min_len

for k in keys_with_optional_tags:
    flexem = structurized_data[k]
    mandatory_tags = get_num_of_mandatory_tags(flexem)
    for el in flexem:
        if len(el['tags'])>mandatory_tags:
            el['tags'] = el['tags'][:-1]

In [89]:
# Another approach: here we attempt to remove one type of tags to get rid of small sets
from queue import *
for k in structurized_data:
    t = structurized_data[k].copy()
    q = PriorityQueue()

    t = sorted(t, key=lambda x: x['card'])

    
    p = t[0]
    fin = []
    while(p['card']<1000 and len(t)>0):
        p = t[0]
        best = (0,0, None, None)
        for i in range(1, len(t)):
            r = t[i]
            pr_tags = intersection(p['tags'], r['tags'])
            pr_card = p['card'] + r['card']
            intersection_size = len(pr_tags)
            if(intersection_size > best[0]):
                best = (intersection_size, pr_card, {'tags': pr_tags, 'card': pr_card}, i)
            elif (intersection_size == best[0] and pr_card <= best[1]):
                best = (intersection_size, pr_card, {'tags': pr_tags, 'card': pr_card}, i)
        if best[3] is not None:
            del(t[best[3]])
            del(t[0])
    
            t.append(best[2])
            t = sorted(t, key=lambda x: x['card'])
    
        else:
            fin += [t[0]]
            del(t[0])
    

    
    t += fin
    print('---------------------')
    print(k)
    for _ in t:
        print(_)
    

---------------------
INTERP
{'tags': [], 'card': 223499}
---------------------
QUB
{'tags': ['WOK'], 'card': 31}
{'tags': ['NWOK'], 'card': 35}
{'tags': [], 'card': 68402}
---------------------
CONJ
{'tags': [], 'card': 44953}
---------------------
FIN
{'tags': ['PERF', 'PRI'], 'card': 1512}
{'tags': ['PL', 'PRI', 'IMPERF'], 'card': 2598}
{'tags': ['SEC'], 'card': 3166}
{'tags': ['PERF', 'TER'], 'card': 5079}
{'tags': ['SG', 'PRI', 'IMPERF'], 'card': 7302}
{'tags': ['PL', 'TER', 'IMPERF'], 'card': 9599}
{'tags': ['SG', 'TER', 'IMPERF'], 'card': 30242}
---------------------
SUBST
{'tags': ['M1'], 'card': 1376}
{'tags': ['VOC', 'SG'], 'card': 1421}
{'tags': ['SG', 'DAT', 'M1'], 'card': 1432}
{'tags': ['PL', 'INST', 'M3'], 'card': 1435}
{'tags': ['DAT', 'PL'], 'card': 1616}
{'tags': ['PL', 'ACC', 'M1'], 'card': 1624}
{'tags': ['M2', 'SG'], 'card': 2065}
{'tags': ['PL', 'LOC', 'N'], 'card': 2169}
{'tags': ['DAT', 'SG'], 'card': 2228}
{'tags': ['INST', 'PL'], 'card': 2233}
{'tags': ['PL', 